In [67]:
import boto3
import re, sys, math
import math
import json, os
import sys
import sagemaker, urllib.request
from sagemaker import get_execution_role
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [68]:
from IPython.display import Image
from IPython.display import display
from time import gmtime, strftime
from sagemaker.predictor import csv_serializer

In [69]:
import pandas as pd

bucket='cosc4assign'

file_key = 'mnist_test.csv'

s3uri = 's3://{}/{}'.format(bucket, file_key)

df_test = pd.read_csv(s3uri)

df_test.head()

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
bucket='cosc4assign'

file_key = 'mnist_train.csv'

s3uri = 's3://{}/{}'.format(bucket, file_key)

df_train = pd.read_csv(s3uri)

df_train.head()

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
train = df_train


In [121]:
import io

s3_resource = boto3.Session().resource('s3')
prefix='assignment_4_cosc'
train_file = 'mnist_train.csv'
def upload_s3_csv(filename, folder, dataframe):
    csv_buffer = io.StringIO()
    dataframe.to_csv(csv_buffer, header=False, index=False)
    s3_resource.Bucket(bucket_name).Object(os.path.join(prefix, folder, filename)).put(Body=csv_buffer.getvalue())

In [122]:
upload_s3_csv(train_file, 'train', train)

In [123]:
from sagemaker.amazon.amazon_estimator import get_image_uri

In [124]:
bucket_name='cosc4assign'
train_file = 'mnist_train.csv'
prefix='assignment_4_cosc'

training_df = sagemaker.inputs.TrainingInput(s3_data="s3://{}/{}/train/".format(bucket_name, prefix, train_file), content_type='text/csv')

In [125]:
knn = sagemaker.estimator.Estimator(
        get_image_uri(boto3.Session().region_name, "knn"),
        get_execution_role(),
        instance_count=1,
        instance_type="ml.m4.xlarge",
        output_path="s3://{}/{}/output/".format(bucket_name,prefix),
        sagemaker_session=sagemaker.Session()
    )

The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [126]:
knn.set_hyperparameters(feature_dim= 784, k= 10, sample_size= 200000, predictor_type= "classifier")

In [127]:
knn.fit({'train': training_df})

2022-11-23 03:56:42 Starting - Starting the training job...
2022-11-23 03:57:06 Starting - Preparing the instances for trainingProfilerReport-1669175802: InProgress
.........
2022-11-23 03:58:25 Downloading - Downloading input data...
2022-11-23 03:59:06 Training - Downloading the training image.....................
2022-11-23 04:02:37 Training - Training image download completed. Training in progress..Docker entrypoint called with argument(s): train
Running default environment configuration script
[11/23/2022 04:02:39 INFO 140068694017856] Reading default configuration from /opt/amazon/lib/python3.7/site-packages/algorithm/resources/default-conf.json: {'_kvstore': 'dist_async', '_log_level': 'info', '_num_gpus': 'auto', '_num_kv_servers': '1', '_tuning_objective_metric': '', '_faiss_index_nprobe': '5', 'epochs': '1', 'feature_dim': 'auto', 'faiss_index_ivf_nlists': 'auto', 'index_metric': 'L2', 'index_type': 'faiss.Flat', 'mini_batch_size': '5000', '_enable_profiler': 'false'}
[11/23/

In [130]:
import time
EndpointName = 'Assign4endd'

In [131]:
knn_predictor = knn.deploy(initial_instance_count=1,serializer = sagemaker.serializers.CSVSerializer(), instance_type='ml.m4.xlarge', endpoint_name=EndpointName )

--------------!

In [132]:
test_batch = df_test.iloc[6:7,1:]
test_batch.head()

,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,1x10,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [133]:
batch_X_csv_buffer = io.StringIO()
test_batch.to_csv(batch_X_csv_buffer, header=False, index=False)
final_batch = batch_X_csv_buffer.getvalue()
print(final_batch)

0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,22,192,134,32,0,0,0,0,0,0,0,0,15,77,5,0,0,0,0,0,0,0,0,0,0,0,0,17,235,250,169,0,0,0,0,0,0,0,0,15,220,241,37,0,0,0,0,0,0,0,0,0,0,0,20,189,253,147,0,0,0,0,0,0,0,0,0,139,253,100,0,0,0,0,0,0,0,0,0,0,0,0,70,253,253,21,0,0,0,0,0,0,0,0,43,254,173,13,0,0,0,0,0,0,0,0,0,0,0,22,153,253,96,0,0,0,0,0,0,0,0,43,231,254,92,0,0,0,0,0,0,0,0,0,0,0,0,163,255,204,11,0,0,0,0,0,0,0,0,104,254,158,0,0,0,0,0,0,0,0,0,0,0,0,0,162,253,178,5,0,0,0,0,0,0,9,131,237,253,0,0,0,0,0,0,0,0,0,0,0,0,0,0,162,253,253,191,175,70,70,70,70,133,197,253,253,169,0,0,0,0,0,0,0,0,0,0,0,0,0,0,51,228,253,253,254,253,253,253,253,254,253,253,219,35,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,17,65,137,254,232,137,137,137,44,253,253,161,0,0,0,0,0,0,0,0,0,0,0,0,

In [134]:
testing_attempt = knn_predictor.predict(final_batch)
testing_attempt

b'{"predictions": [{"predicted_label": 4.0}]}'

In [150]:
from PIL import Image
import requests
from io import BytesIO

url = 'https://datasets-server.huggingface.co/assets/mnist/--/mnist/test/22/image/image.jpg'
response = requests.get(url)
image_rep = Image.open(BytesIO(response.content))
image_rep = np.array(image_rep.getdata(), dtype = np.uint8).reshape(image_rep.size[0], image_rep.size[1])

In [151]:
testing_data = ','.join([ str(image_rep[i,j]) for i in range(28) for j in range(28)])

In [152]:
knn_predictor.predict(testing_data)

b'{"predictions": [{"predicted_label": 6.0}]}'